In [70]:
import os, json
import zipfile
import time

In [71]:
class Printer:
    params_file_name = 'params.json'
    
    def __init__(self):
        pass

    def set_params_file(self):
        if not os.path.exists(self.params_file_name):
            print('Criando arquivo de parâmetros\n')
        
            with open(self.params_file_name, 'w') as file:
                json.dump({'printer_ip':''}, file)
            
            print('Arquivo criado, preencha a porta da impressora na rede local e então pressione ENTER para prosseguir')
            input()

    @property
    def printer_port(self):
        return self.get_printer_port

    @printer_port.getter
    def get_printer_port(self):
        with open(self.params_file_name,'r') as file:
            json_dict = json.load(file)
        
        printerPort = json_dict['printer_ip']
        return printerPort

    def print_file(self, file_path):
        print_command = f'copy "{file_path}" {self.printer_port}'
        os.system(print_command)
        # print(print_command) # use for debugging

In [73]:
# sets printer to be used
printer = Printer()
printer.set_params_file()

# create labels folder
labels_folder_name = 'Etiquetas'
os.makedirs(labels_folder_name, exist_ok=True)

cwd = os.getcwd()
labels_folder_path = os.path.join(cwd, labels_folder_name)

In [74]:
def filter_by_file_type(file_list, extension):
    extension = '.' + extension 
    file_type_verifier = lambda x: os.path.splitext(x)[1] == extension
    filtered_list = list(filter(file_type_verifier, file_list))
    return filtered_list


def prints_file_on_labels_folder(file):
    """Get the full path to a file and send a command to the printer, then deletes it."""

    file_path = os.path.join(labels_folder_path, file)
    Printer().print_file(file_path)
    
    os.remove(file_path)

    print(f'Printed: {file}\n')
    print('-----------------------------------------------------------\n\n')

In [75]:
os.system('cls')

loop = True

while loop:
    time.sleep(1)
    
    pendingJobsFiles = os.listdir(labels_folder_name)
    txt_files_list = filter_by_file_type(pendingJobsFiles, 'txt')
    zip_files_list = filter_by_file_type(pendingJobsFiles, 'zip')

    # executes only if there are txt files in the folder, else it's an empty list
    list(map(prints_file_on_labels_folder, txt_files_list))
    
    # executes only once for each loop
    if zip_files_list:
        file_path = os.path.join(labels_folder_path, zip_files_list[0])

        with zipfile.ZipFile(file_path,'r') as zip_ref:
            zip_ref.extractall(labels_folder_name)
        
        os.remove(file_path)

        print(f'unziped: {zip_files_list[0]}\n')
         
